<a href="https://colab.research.google.com/github/alessandromenchetti/COMS6998-Final-Project/blob/main/LLM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initial Setup

Importing the relevant datasets and modules from HuggingFace and Python. We create the reference files and save them to the drive. We also craft the chain-of-thought prompt that will be utilized in some form throughout. Finally, we call the OpenAI API to run the bulk evaluation using the prompt and store these as a proxy for ground truths.

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset, Dataset
import random
import pandas as pd

# Load the dataset. If the dataset has a known split, specify it, otherwise omit 'split'.
dataset = load_dataset("talkmap/banking-conversation-corpus", split="train")

# Extract all unique conversation IDs
unique_conversation_ids = dataset.unique("conversation_id")

# Shuffle the conversation IDs
random.shuffle(unique_conversation_ids)

# Select the desired number of conversations for training and testing
conversations = set(unique_conversation_ids[:5100])

subset = dataset.filter(lambda example: example["conversation_id"] in conversations)

subset = subset.map(lambda example: {
    "formatted_line": f"{example['speaker']}: {example['text']}\n"
})

# Step 2: Sort the dataset by date_time to ensure chronological order
subset = subset.sort("date_time")

# Step 3: Convert the dataset to a Pandas DataFrame for grouping
df = subset.to_pandas()

# Step 4: Group by conversation_id and concatenate all formatted lines
grouped_df = df.groupby("conversation_id")["formatted_line"].apply(lambda lines: "".join(lines)).reset_index()

# The grouped_df now has one row per conversation_id and a single column with the entire conversation.
# Let's rename the concatenated column to something more descriptive, like "conversation_text"
grouped_df = grouped_df.rename(columns={"formatted_line": "conversation_text"})

# Step 5: Convert back to a Dataset
new_dataset = Dataset.from_pandas(grouped_df, preserve_index=False)

Filter:   0%|          | 0/5532112 [00:00<?, ? examples/s]

Map:   0%|          | 0/93600 [00:00<?, ? examples/s]

In [ ]:
print(grouped_df.at[5002, 'conversation_text'])

agent: Hello, thank you for holding. My name is Neil and and I'll be happy to assist you today. Can you please provide me with your name and account number so I can pull up your information?
agent: Great, Mammie. I just need to verify some personal to ensure we'm speaking with the correct person. Can you please confirm your date of birth?
client: Sure, it's 02/05/1990.
agent: Perfect, Mammie. How can I assist you today?
client: Well, I called last week about setting up a tax-advantaged account, but the representative I spoke with couldn't help me. They told me I needed to call a different number, but I'm not sure which one that CanNeil: I apologize for the confusion, Mammie. Let me see if I can do to help. Can you tell me a little more about what you're looking for in a tax-advantaged account? Are you interested in a specific type, account, such as a 401(k) or an IRA?
client: Yes, I'm interested in an IRA. I want to make sure I'm taking advantage of all the tax benefits I can get.
agen

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
new_dataset.save_to_disk("/content/drive/MyDrive/LLM_Project_Data/new_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/5100 [00:00<?, ? examples/s]

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall

  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached exceptiongroup-1.2.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.me

In [ ]:
import os
import random
import openai
from openai import OpenAI
from google.colab import userdata

from datasets import load_from_disk
apiKey  = userdata.get('OPENAI_API_KEY')
# Set your OpenAI API key.

#openai.api_key = apiKey
client = OpenAI(api_key=apiKey)
# Load the dataset you previously saved
new_dataset = load_from_disk("/content/drive/MyDrive/LLM_Project_Data/new_dataset")

# Convert to a list to easily pick a random conversation
conv_list = list(new_dataset)

# Select a random conversation entry
random_conv = random.choice(conv_list)
conversation_id = random_conv["conversation_id"]
conversation_text = random_conv["conversation_text"]

# Prepare the system and user prompts
system_prompt = """You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to predict future revenue impact.
Review the conversation transcript below and follow these steps:

1.) Identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

DIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.

2.) Identify and analyze indirect revenue indicators. These are more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).

Identify and extract relevant parts of the text corresponding to these indirect revenue indicators and put them into the format:

INDIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO INDIRECT REVENUE INDICATORS FOUND.

3.) Identify and analyze risk factors. These are elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

Identify and extract relevant parts of the text corresponding to these risk factors and put them into the format:

RISK FACTORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO RISK FACTORS FOUND.

4.) Make an assessment of the weight and reliability of the evidence found related to direct revenue indicators, indirect revenue indicators, and risk factors. Distinguish between explicit statements and implied meanings. Consider the quantity, consistency, and strength of the indicators identified. Put your findings into the format:

EVIDENCE STRENGTH ASSESSMENT:
- Direct Revenue Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]
- Indirect Revenue Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]
- Risk Factor Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]

5.) Based on the evidence strength assessment, estimate whether the bank is most likely to see more, less, or the same amount of revenue from that client in the future and the degree of confidence in that revenue trajectory forecast.

You can think about the revenue trajectories as follows:
• More Revenue: Clear indicators of expansion, positive satisfaction, life events leading to growth.
• Same Revenue: Stable relationship, resolved issues, no major changes indicated.
• Less Revenue: Risk factors present, reduction in services, competitor interest.

You can think about confidence in your revenue trajectory forecast as follows:
• High: Multiple consistent indicators across categories.
• Medium: Some clear indicators but with gaps or inconsistencies.
• Low: Limited information or conflicting indicators.

Output your final determination in the following format:

REVENUE PREDICTION:
Direction: [More/Same/Less] revenue expected
Confidence Level: [High/Medium/Low]
Primary Drivers: [List top 2-3 factors]
"""

# The user prompt includes the conversation_id and its transcript
user_prompt = f"Here is the conversation transcript:\n\n{conversation_text}"

# Make the API call using ChatCompletion (for gpt-4 or gpt-3.5-turbo)
try:
    response = client.chat.completions.create(
        model="gpt-4o",  # Change to "gpt-3.5-turbo" if gpt-4 access is not available
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0
    )
    print(response.choices[0].message.content)

except openai.error.OpenAIError as e:
    print(f"An error occurred: {e}")
    '''
response = client.chat.completions.create(
    model="gpt-4o",  # or "gpt-3.5-turbo", depending on your available model
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.0  # lower temperature for more deterministic results
)

# Print the LLM's response
print(response.choices[0].message.content)
'''


DIRECT REVENUE INDICATORS:
- "I think I'd like to donate $100.": This indicates a one-time transaction of $100, which is a direct revenue contribution to the bank through transaction fees or service charges associated with processing the donation.

INDIRECT REVENUE INDICATORS:
- "Just a one-time donation for now, thank you.": This suggests that the client is not currently interested in setting up a recurring transaction, which could have indicated a more stable, ongoing revenue stream.
- "(sighs) Okay, my address is 123 Main Street, and my phone number is 555-555-5555.": The sigh and tone suggest possible frustration or impatience, which could impact future interactions and satisfaction levels.

RISK FACTORS:
- "Can we just get this over with already?": This indicates impatience or dissatisfaction with the process, which could be a risk factor for future interactions or continued business with the bank.
- "Yeah, yeah. Thanks, I guess.": The dismissive tone suggests a lack of enthusiasm

In [ ]:
#Run Batch on OpenAI

df = new_dataset.to_pandas()
queryList = []

for index, row in df.iterrows():
  query = {"custom_id": row['conversation_id'], "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4o-mini", "messages": [{"role": "system", "content": system_prompt},{"role": "user", "content": f"Here is the conversation transcript:\n\n{row['conversation_text']}"}],"max_tokens": 5000}}
  queryList.append(query)

#

In [ ]:
import json
file_name = "/content/drive/MyDrive/batch_request.jsonl"

with open(file_name, 'w') as file:
    for obj in queryList:
        file.write(json.dumps(obj) + '\n')

In [ ]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)
print(batch_file)

FileObject(id='file-QEkVdh1eBEgFJs8XNm1Vvw', bytes=34903389, created_at=1733635025, filename='batch_request.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [ ]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)
print(batch_job.id)

batch_67552aefc7708191815bbdcc6d598eef


In [ ]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)
#Code to finish is here: https://cookbook.openai.com/examples/batch_processing

Batch(id='batch_67552aefc7708191815bbdcc6d598eef', completion_window='24h', created_at=1733634799, endpoint='/v1/chat/completions', input_file_id='file-T1xne9vc8udZToLeU3irJF', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1733636994, error_file_id=None, errors=None, expired_at=None, expires_at=1733721199, failed_at=None, finalizing_at=1733636538, in_progress_at=1733634801, metadata=None, output_file_id='file-MyrgUe1zX3k4iS4iU3noBu', request_counts=BatchRequestCounts(completed=5100, failed=0, total=5100))


In [ ]:
result_file_name = "/content/drive/MyDrive/batch_results.jsonl"
'''
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

with open(result_file_name, 'wb') as file:
    file.write(result)
'''

"\nresult_file_id = batch_job.output_file_id\nresult = client.files.content(result_file_id).content\n\nwith open(result_file_name, 'wb') as file:\n    file.write(result)\n"

In [ ]:

results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())

        results.append(json_object)

print(results[0])

NameError: name 'json' is not defined

In [ ]:
import pandas as pd
ids = []
responses = []

for result in results:
  id = result['custom_id']
  response = result['response']['body']['choices'][0]['message']['content']
  ids.append(id)
  responses.append(response)

truthsDF = pd.DataFrame({'conversation_id': ids, 'response': responses})
truthsDF.to_csv('/content/drive/MyDrive/LLM_Project_Data/truths.csv', index=False)

KeyError: 'custom_id'

In [ ]:
truthsDF = pd.read_csv('/content/drive/MyDrive/LLM_Project_Data/truths.csv')
print(truthsDF.head())

                    conversation_id  \
0  0004a37093f146a29c6b7ba63866fbaa   
1  00054abe1b934c4e989fdc31cf0d75f3   
2  0021a1721d21445da3f9b07bd4df532d   
3  00227ba3aad44e0da210ba14c72df1c2   
4  0027a84b27bd40c9b63e921db21b17a1   

                                            response  
0  DIRECT REVENUE INDICATORS:\n- "I need to chang...  
1  DIRECT REVENUE INDICATORS:\n- "I'm interested ...  
2  1.) DIRECT REVENUE INDICATORS:\nNO DIRECT REVE...  
3  DIRECT REVENUE INDICATORS:\n- "I'm going to go...  
4  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...  


In [ ]:
import re
import pandas as pd

# Example DataFrame:
# df = pd.DataFrame({
#     "conversation_id": [...],
#     "response_text": ["FULL RESPONSE TEXT HERE", ...]
# })

# Compile regex patterns for efficiency
direct_pattern = re.compile(
    r'(?s)DIRECT REVENUE INDICATORS:(.*?)(?:INDIRECT REVENUE INDICATORS:|RISK FACTORS:|EVIDENCE STRENGTH ASSESSMENT:|REVENUE PREDICTION:|$)'
)
indirect_pattern = re.compile(
    r'(?s)INDIRECT REVENUE INDICATORS:(.*?)(?:RISK FACTORS:|EVIDENCE STRENGTH ASSESSMENT:|REVENUE PREDICTION:|$)'
)
risk_pattern = re.compile(
    r'(?s)RISK FACTORS:(.*?)(?:EVIDENCE STRENGTH ASSESSMENT:|REVENUE PREDICTION:|$)'
)
evid_pattern = re.compile(
    r'(?s)EVIDENCE STRENGTH ASSESSMENT:(.*?)(?:REVENUE PREDICTION:|$)'
)
prediction_pattern = re.compile(
    r'(?s)REVENUE PREDICTION:(.*)$'
)
direction_pattern = re.compile(
    r'Direction:\s*(More|Same|Less)', re.IGNORECASE
)

def parse_response(text):
    # Extract sections using the regex patterns
    direct_match = direct_pattern.search(text)
    indirect_match = indirect_pattern.search(text)
    risk_match = risk_pattern.search(text)
    evid_match = evid_pattern.search(text)
    prediction_match = prediction_pattern.search(text)

    direct_indicators = direct_match.group(1).strip() if direct_match else None
    indirect_indicators = indirect_match.group(1).strip() if indirect_match else None
    risk_factors = risk_match.group(1).strip() if risk_match else None
    evid_assessment = evid_match.group(1).strip() if evid_match else None
    prediction = prediction_match.group(1).strip() if prediction_match else None

    # Extract direction from the prediction section if available
    direction_match = direction_pattern.search(prediction) if prediction else None
    direction = direction_match.group(1) if direction_match else None

    return pd.Series({
        "DIRECT_REVENUE_INDICATORS": direct_indicators if direct_indicators else "NO DIRECT REVENUE INDICATORS FOUND",
        "INDIRECT_REVENUE_INDICATORS": indirect_indicators if indirect_indicators else "NO INDIRECT REVENUE INDICATORS FOUND",
        "RISK_FACTORS": risk_factors if risk_factors else "NO RISK FACTORS FOUND",
        "EVIDENCE_STRENGTH_ASSESSMENT": evid_assessment if evid_assessment else "NO EVIDENCE STRENGTH ASSESSMENT FOUND",
        "REVENUE_PREDICTION": prediction,
        "REVENUE_DIRECTION": direction
    })

# Apply the function to the dataframe
truthsDF[["DIRECT_REVENUE_INDICATORS", "INDIRECT_REVENUE_INDICATORS", "RISK_FACTORS", "EVIDENCE_STRENGTH_ASSESSMENT", "REVENUE_PREDICTION", "REVENUE_DIRECTION"]] = truthsDF["response"].apply(parse_response)
truthsDF.to_csv('/content/drive/MyDrive/LLM_Project_Data/truths_expanded2.csv', index=False)

#Now let's download Llama 3.2 3b

In [ ]:
!tar -xvzf /content/drive/MyDrive/LLM_Project_Data/llama-3.2-transformers-3b-instruct-v1.tar.gz

tar: /content/drive/MyDrive/LLM_Project_Data/llama-3.2-transformers-3b-instruct-v1: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Now that llama base model is uploaded, we test it.

In [ ]:
import torch
from transformers  import AutoTokenizer, AutoModelForCausalLM

# Path to your local LLaMA 3B model. This directory should contain the model files.
model_path = "/content/drive/MyDrive/LLM_Project_Data/llama3.2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model (make sure you have enough GPU RAM, else run on CPU or a smaller model)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")  # device_map="auto" to run on GPU if available

full_prompt = "What is the capitol of France?"

# Tokenize the input
inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

# Generate a response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.0,
        do_sample=False,  # deterministic output
    )

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is the capitol of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
What is the capital of France? Paris
W

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
#Set some global variables
from datasets import load_from_disk, Dataset
import random
import pandas as pd

# Load the dataset. If the dataset has a known split, specify it, otherwise omit 'split'.
dataset = load_from_disk("/content/drive/MyDrive/LLM_Project_Data/new_dataset")

dataset = dataset.shuffle(seed=42)

# Perform the train/test split, taking 100 rows for the test set
split_dataset = dataset.train_test_split(test_size=100)

# Access the train and test subsets
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

# Optionally, you can save them or further process them
train_dataset.save_to_disk("/content/drive/MyDrive/LLM_Project_Data/train_dataset")
test_dataset.save_to_disk("/content/drive/MyDrive/LLM_Project_Data/test_dataset")

print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Train dataset size: 5000
Test dataset size: 100


In [ ]:
trainDF = train_dataset.to_pandas()
testDF = test_dataset.to_pandas()

print(testDF.head())

                    conversation_id  \
0  aee2f63e653049769bda7b0c99ad3b90   
1  9dff2275fe564ab19a4d785f2cb4f7bf   
2  46607d7eb97942eeb48cef087a0dceaf   
3  0952bea524014a8b9a08af4162aea9f8   
4  44f9839702f8459885cac630945ffe21   

                                   conversation_text  
0  agent: Good morning, thank you for calling Uni...  
1  agent: Good morning, thank you for calling Uni...  
2  agent: Hello morning, thank you for calling Un...  
3  agent: Hello, thank you for holding Union Fina...  
4  agent: Good afternoon, thank you for calling U...  


In [ ]:
!mv /content/LICENSE.txt /content/README.md /content/USE_POLICY.md /content/config.json /content/model-00001-of-00002.safetensors /content/model-00002-of-00002.safetensors /content/model.safetensors.index.json /content/special_tokens_map.json /content/tokenizer.json /content/tokenizer_config.json /content/drive/MyDrive/LLM_Project_Data/llama3.2

In [ ]:
system_prompt = """You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to predict future revenue impact.
Review the conversation transcript below and follow these steps:

1.) Identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

DIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.

2.) Identify and analyze indirect revenue indicators. These are more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).

Identify and extract relevant parts of the text corresponding to these indirect revenue indicators and put them into the format:

INDIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO INDIRECT REVENUE INDICATORS FOUND.

3.) Identify and analyze risk factors. These are elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

Identify and extract relevant parts of the text corresponding to these risk factors and put them into the format:

RISK FACTORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO RISK FACTORS FOUND.

4.) Make an assessment of the weight and reliability of the evidence found related to direct revenue indicators, indirect revenue indicators, and risk factors. Distinguish between explicit statements and implied meanings. Consider the quantity, consistency, and strength of the indicators identified. Put your findings into the format:

EVIDENCE STRENGTH ASSESSMENT:
- Direct Revenue Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]
- Indirect Revenue Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]
- Risk Factor Evidence: [Strong/Moderate/Weak]
  Key points: [Summary]

5.) Based on the evidence strength assessment, estimate whether the bank is most likely to see more, less, or the same amount of revenue from that client in the future and the degree of confidence in that revenue trajectory forecast.

You can think about the revenue trajectories as follows:
• More Revenue: Clear indicators of expansion, positive satisfaction, life events leading to growth.
• Same Revenue: Stable relationship, resolved issues, no major changes indicated.
• Less Revenue: Risk factors present, reduction in services, competitor interest.

You can think about confidence in your revenue trajectory forecast as follows:
• High: Multiple consistent indicators across categories.
• Medium: Some clear indicators but with gaps or inconsistencies.
• Low: Limited information or conflicting indicators.

Output your final determination in the following format:

REVENUE PREDICTION:
Direction: [More/Same/Less] revenue expected
Confidence Level: [High/Medium/Low]
Primary Drivers: [List top 2-3 factors]
"""

referenceDF = pd.read_csv('/content/drive/MyDrive/LLM_Project_Data/truths_expanded.csv')
print(referenceDF.head())

                    conversation_id  \
0  0004a37093f146a29c6b7ba63866fbaa   
1  00054abe1b934c4e989fdc31cf0d75f3   
2  0021a1721d21445da3f9b07bd4df532d   
3  00227ba3aad44e0da210ba14c72df1c2   
4  0027a84b27bd40c9b63e921db21b17a1   

                                            response  \
0  DIRECT REVENUE INDICATORS:\n- "I need to chang...   
1  DIRECT REVENUE INDICATORS:\n- "I'm interested ...   
2  1.) DIRECT REVENUE INDICATORS:\nNO DIRECT REVE...   
3  DIRECT REVENUE INDICATORS:\n- "I'm going to go...   
4  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...   

                           DIRECT_REVENUE_INDICATORS  \
0  - "I need to change the name on my account.": ...   
1  - "I'm interested in learning more about your ...   
2         NO DIRECT REVENUE INDICATORS FOUND.\n\n2.)   
3  - "I'm going to go ahead and order a new card ...   
4              - NO DIRECT REVENUE INDICATORS FOUND.   

                         INDIRECT_REVENUE_INDICATORS  \
0  - "Well, I recently got married.

In [ ]:
referenceDF = referenceDF.set_index('conversation_id')
print(referenceDF.head())

                                                                           response  \
conversation_id                                                                       
0004a37093f146a29c6b7ba63866fbaa  DIRECT REVENUE INDICATORS:\n- "I need to chang...   
00054abe1b934c4e989fdc31cf0d75f3  DIRECT REVENUE INDICATORS:\n- "I'm interested ...   
0021a1721d21445da3f9b07bd4df532d  1.) DIRECT REVENUE INDICATORS:\nNO DIRECT REVE...   
00227ba3aad44e0da210ba14c72df1c2  DIRECT REVENUE INDICATORS:\n- "I'm going to go...   
0027a84b27bd40c9b63e921db21b17a1  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...   

                                                          DIRECT_REVENUE_INDICATORS  \
conversation_id                                                                       
0004a37093f146a29c6b7ba63866fbaa  - "I need to change the name on my account.": ...   
00054abe1b934c4e989fdc31cf0d75f3  - "I'm interested in learning more about your ...   
0021a1721d21445da3f9b07bd4df532d         N

In [ ]:
#Run query on random test convo to get params right
convo = testDF.at[0,'conversation_text']

import torch
from transformers  import AutoTokenizer, AutoModelForCausalLM

# Path to your local LLaMA 3B model. This directory should contain the model files.
model_path = "/content/drive/MyDrive/LLM_Project_Data/llama3.2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model (make sure you have enough GPU RAM, else run on CPU or a smaller model)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")  # device_map="auto" to run on GPU if available

messages =  [
  {"role": "system", "content": system_prompt},
  {"role": "user", "content": f"Here is the conversation transcript:\n\n{convo}"}
]



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to predict future revenue impact.
Review the conversation transcript below and follow these steps:

1.) Identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

DIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
- ...

If none are found, output: NO DIRECT REVENUE INDICATORS FOUN

In [ ]:


# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate a response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=5000,
        temperature=0.0,
        do_sample=False,  # deterministic output
    )

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


system

You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to predict future revenue impact.
Review the conversation transcript below and follow these steps:

1.) Identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

DIRECT REVENUE INDICATORS:
- [Quote from transcript]: [Brief explanation of relevance]
-...

If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.

2.) Identify and analyze indirect revenue indicato

In [ ]:
import re
prediction_pattern = re.compile(
    r'(?s)REVENUE PREDICTION:(.*)$'
)
direction_pattern = re.compile(
    r'Direction:\s*(More|Same|Less)', re.IGNORECASE
)


prediction_match = prediction_pattern.search(generated_text)


prediction = prediction_match.group(1).strip() if prediction_match else None

    # Extract direction from the prediction section if available
direction_match = direction_pattern.search(prediction) if prediction else None
direction = direction_match.group(1) if direction_match else None

print(direction)

id = convo = testDF.at[0,'conversation_id']

groundTruth = referenceDF.at[id,'REVENUE_DIRECTION']
print(groundTruth)


More
Same


#Integrated Code For Benchmark Local Performance
This block of code takes a lot of the components experimented with above and integrates them into a seamless run and evaluation of performance on the test set from the openAI results.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import time
import pandas as pd

# Ensure referenceDF is indexed by conversation_id if necessary
# referenceDF = referenceDF.set_index("conversation_id")

# Path to your local LLaMA 3B model
model_path = "/content/drive/MyDrive/LLM_Project_Data/llama3.2"

# Load the tokenizer and model once
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# Compile regex patterns
prediction_pattern = re.compile(r'(?s)REVENUE PREDICTION:(.*)$')
direction_pattern = re.compile(r'Direction:\s*(More|Same|Less)', re.IGNORECASE)

results = []

for idx, row in testDF.iterrows():
    conversation_id = row["conversation_id"]
    convo = row["conversation_text"]

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Here is the conversation transcript:\n\n{convo}"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    start_time = time.time()
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    end_time = time.time()

    inference_time = end_time - start_time

    # Extract generated tokens count
    output_ids = output[0]
    input_ids = inputs["input_ids"][0]
    generated_token_count = len(output_ids) - len(input_ids)

    # Calculate tokens per second
    if inference_time > 0 and generated_token_count > 0:
        tokens_per_second = generated_token_count / inference_time
    else:
        tokens_per_second = 0.0

    generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)

    # Extract prediction section
    prediction_match = prediction_pattern.search(generated_text)
    prediction = prediction_match.group(1).strip() if prediction_match else None

    # Extract direction
    direction_match = direction_pattern.search(prediction) if prediction else None
    predicted_direction = direction_match.group(1) if direction_match else None

    # Ground truth direction from referenceDF
    ground_truth_direction = referenceDF.at[conversation_id, "REVENUE_DIRECTION"] if conversation_id in referenceDF.index else None

    results.append({
        "conversation_id": conversation_id,
        "predicted_direction": predicted_direction,
        "ground_truth_direction": ground_truth_direction,
        "tokens_per_second": tokens_per_second
    })

results_df = pd.DataFrame(results)
results_df.to_csv("/content/drive/MyDrive/LLM_Project_Data/baseline_performance.csv", index=False)

# Calculate accuracy
correct = (results_df["predicted_direction"] == results_df["ground_truth_direction"]).sum()
total = len(results_df)
accuracy = correct / total if total > 0 else 0.0

# Calculate average tokens/second
average_throughput = results_df["tokens_per_second"].mean()

print(f"Accuracy: {accuracy:.2%}")
print(f"Average Tokens/Second: {average_throughput:.2f}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for ope

Accuracy: 64.00%
Average Tokens/Second: 31.03


In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import classification_report

y_true = results_df["ground_truth_direction"]
y_pred = results_df["predicted_direction"]

print(classification_report(y_true, y_pred, labels=["More", "Same", "Less"], zero_division=0))


              precision    recall  f1-score   support

        More       0.61      0.86      0.71        29
        Same       0.70      0.38      0.49        42
        Less       0.64      0.79      0.71        29

    accuracy                           0.64       100
   macro avg       0.65      0.68      0.64       100
weighted avg       0.65      0.64      0.62       100



#Initial Results
As can be seen above, we are working with a baseline accuracy rate of about 64% as our benchmark. And throughput of 31 tokens/s. Ample room for improvement!

Our next step will be to break the chain-of-thought into five discrete prompts, and build LoRA adapters for each prompt. The hope is that chaining these fine-tuned adapted models will beef up accuracy.

#Fine Tuning

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install peft
!pip install bitsandbytes
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import load_from_disk
import pandas as pd

# Load the dataset. If the dataset has a known split, specify it, otherwise omit 'split'.

train_dataset = load_from_disk("/content/drive/MyDrive/LLM_Project_Data/train_dataset")
test_dataset = load_from_disk("/content/drive/MyDrive/LLM_Project_Data/test_dataset")
trainDF = train_dataset.to_pandas()
truthsDF = pd.read_csv('/content/drive/MyDrive/LLM_Project_Data/truths_expanded2.csv')

In [ ]:
tuneDF = trainDF.merge(truthsDF, on='conversation_id', how='inner')
print(tuneDF.head())

                    conversation_id  \
0  86f87e8de3d74e6fa7a011b6cea06399   
1  f584466524b74883933182a1e1a5c086   
2  c080af549e6046b992d21b7c4a000d3d   
3  4d6c94014f2147fd9323c560bc63d277   
4  1144df3f352c47b0b213891fe1d9b2b2   

                                   conversation_text  \
0  agent: Hello, thank you for calling Union Fina...   
1  agent: Good morning, thank you for calling Uni...   
2  agent: Good morning, thank you for calling Uni...   
3  agent: Hello morning, thank you for calling Un...   
4  agent: Thank you for calling Union Financial. ...   

                                            response  \
0  DIRECT REVENUE INDICATORS:\nNO DIRECT REVENUE ...   
1  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...   
2  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...   
3  1.) DIRECT REVENUE INDICATORS:\n- NO DIRECT RE...   
4  DIRECT REVENUE INDICATORS:\n- "I'm calling to ...   

                           DIRECT_REVENUE_INDICATORS  \
0                NO DIRECT REVENUE 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import pandas as pd
import re
import time

# Base model path
base_model_path = "/content/drive/MyDrive/LLM_Project_Data/llama3.2"
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
base_model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="cuda")
base_model.to("cuda")
tokenizer.pad_token = tokenizer.eos_token

# We'll define a helper function to create a single sequence and compute labels
def tokenize_causal(example, system_prompt, user_prompt, answer_text, max_length=2048):
    """
    Constructs a single prompt of form:
    system_prompt
    user_prompt
    ### OUTPUT START ###
    answer_text

    Then tokenizes and sets labels so that tokens before ### OUTPUT START ### are -100.
    """
    # Insert the marker
    full_text = system_prompt.strip() + "\n" + user_prompt.strip() + "\n\n### OUTPUT START ###\n" + answer_text.strip()
    #print(full_text)

    model_inputs = tokenizer(full_text, max_length=max_length, truncation=True, padding="max_length")

    input_ids = model_inputs["input_ids"]

    # Find the position of the marker
    # We look for the tokenization of "### OUTPUT START ###"
    # To ensure a robust search, we re-tokenize the marker alone and search that sequence in input_ids.
    marker_ids = tokenizer("### OUTPUT START ###\n", add_special_tokens=False)["input_ids"]

    # Search for marker_ids in input_ids
    # A simple way: scan for a subsequence match
    def find_subsequence(main_list, sub_list):
        sub_len = len(sub_list)
        for i in range(len(main_list)-sub_len+1):
            if main_list[i:i+sub_len] == sub_list:
                return i
        return -1

    marker_pos = find_subsequence(input_ids, marker_ids)
    if marker_pos == -1:
        # If not found, default to no answer portion
        # This means no tokens are labeled, which is unusual. You may choose to handle differently.
        # Here we assume no answer available => all labels = -100
        labels = [-100]*len(input_ids)
    else:
        # Tokens before the marker are context (no supervision)
        # Tokens from marker onwards are target
        labels = [-100]*len(input_ids)
        # Allow model to predict the answer tokens (including the marker line itself if desired)
        # If you don't want the model to reproduce the marker, shift marker_pos+len(marker_ids)
        # Here we include marker in the labels so the model predicts from the marker line onward:
        for j in range(marker_pos, len(input_ids)):
            if input_ids[j] != tokenizer.pad_token_id:
                labels[j] = input_ids[j]

    model_inputs["labels"] = labels
    return model_inputs


def tokenize_function_direct_rev(example):
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

        Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

        DIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.
        '''
    )
    user_content = f"Here is the conversation transcript:\n\n{example['conversation_text']}"
    answer_text = "DIRECT REVENUE INDICATORS:\n" + example["DIRECT_REVENUE_INDICATORS"]
    return tokenize_causal(example, system_content, user_content, answer_text, max_length=1024)

def tokenize_function_indirect_rev(example):
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze indirect revenue indicators. These are more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).

        Identify and extract relevant parts of the text corresponding to these indirect revenue indicators and put them into the format:

        INDIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO INDIRECT REVENUE INDICATORS FOUND.
        '''
    )
    user_content = f"Here is the conversation transcript:\n\n{example['conversation_text']}"
    answer_text = "INDIRECT REVENUE INDICATORS:\n" + example["INDIRECT_REVENUE_INDICATORS"]
    return tokenize_causal(example, system_content, user_content, answer_text, max_length=1024)

def tokenize_function_risk(example):
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze risk factors. These are elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Identify and extract relevant parts of the text corresponding to these risk factors and put them into the format:

        RISK FACTORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO RISK FACTORS FOUND.
        '''
    )
    user_content = f"Here is the conversation transcript:\n\n{example['conversation_text']}"
    answer_text = "RISK FACTORS:\n" + example["RISK_FACTORS"]
    return tokenize_causal(example, system_content, user_content, answer_text, max_length=1024)

def tokenize_function_assess(example):
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a summary of a conversation between a bank client and a service agent and assess the evidence provided to assist in predicting future revenue impact.
        Review the provided summary below, which highlights elements of the conversation relevant to three indicators of future revenue:

        1.) Direct Revenue Indicators: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Indicators: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factors: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        The summary will present evidence relevent to these indicators in the format:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        Use the  summary to make an assessment of the weight and reliability of the evidence found related to direct revenue indicators, indirect revenue indicators, and risk factors. Distinguish between explicit statements and implied meanings. Consider the quantity, consistency, and strength of the indicators identified. Put your findings into the format:

        EVIDENCE STRENGTH ASSESSMENT:
        - Direct Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Indirect Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Risk Factor Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        '''
    )
    user_content = (
        "Here is the summary:\n\n" +
        "DIRECT REVENUE INDICATORS:\n" + example["DIRECT_REVENUE_INDICATORS"] + "\n\n" +
        "INDIRECT REVENUE INDICATORS:\n" + example["INDIRECT_REVENUE_INDICATORS"] + "\n\n" +
        "RISK FACTORS:\n" + example["RISK_FACTORS"]
    )

    answer_text = "EVIDENCE STRENGTH ASSESSMENT:\n" + example["EVIDENCE_STRENGTH_ASSESSMENT"]
    return tokenize_causal(example, system_content, user_content, answer_text, max_length=1024)

def tokenize_function_predict(example):
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze an assessment of a conversation between a bank client and a service agent to predict future revenue impact. Review the provided assessment below, which analyzes evidence relevant to three indicators of future revenue:

        1.) Direct Revenue Evidence: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Evidence: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factor Evidence: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Based on the evidence strength assessment, estimate whether the bank is most likely to see more, less, or the same amount of revenue from that client in the future and the degree of confidence in that revenue trajectory forecast.

        You can think about the revenue trajectories as follows:
        • More Revenue: Clear indicators of expansion, positive satisfaction, life events leading to growth.
        • Same Revenue: Stable relationship, resolved issues, no major changes indicated.
        • Less Revenue: Meaningful factors present, reduction in services, competitor interest.

        You can think about confidence in your revenue trajectory forecast as follows:
        • High: Multiple consistent indicators across categories.
        • Medium: Some clear indicators but with gaps or inconsistencies.
        • Low: Limited information or conflicting indicators.

        Output your final determination in the following format:

        REVENUE PREDICTION:
        Direction: [More/Same/Less] revenue expected
        Confidence Level: [High/Medium/Low]
        Primary Drivers: [List top 2-3 factors]
        '''
    )
    user_content = "Here is the assessment:\n\nEVIDENCE STRENGTH ASSESSMENT:\n" + example["EVIDENCE_STRENGTH_ASSESSMENT"]
    answer_text = "REVENUE PREDICTION:\n" + example["REVENUE_PREDICTION"]
    return tokenize_causal(example, system_content, user_content, answer_text, max_length=1024)


# Assuming tuneDF is defined and contains the necessary columns
train_direct = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "DIRECT_REVENUE_INDICATORS"]])
train_indirect = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "INDIRECT_REVENUE_INDICATORS"]])
train_risk = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "RISK_FACTORS"]])
train_evidence = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text",
                                              "DIRECT_REVENUE_INDICATORS",
                                              "INDIRECT_REVENUE_INDICATORS",
                                              "RISK_FACTORS",
                                              "EVIDENCE_STRENGTH_ASSESSMENT"]])
train_prediction = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text",
                                                 "EVIDENCE_STRENGTH_ASSESSMENT",
                                                 "REVENUE_PREDICTION"]])

tokenized_direct = train_direct.map(tokenize_function_direct_rev, batched=False, remove_columns=train_direct.column_names)
tokenized_indirect = train_indirect.map(tokenize_function_indirect_rev, batched=False, remove_columns=train_indirect.column_names)
tokenized_risk = train_risk.map(tokenize_function_risk, batched=False, remove_columns=train_risk.column_names)
tokenized_evidence = train_evidence.map(tokenize_function_assess, batched=False, remove_columns=train_evidence.column_names)
tokenized_prediction = train_prediction.map(tokenize_function_predict, batched=False, remove_columns=train_prediction.column_names)

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

layer = ['direct', 'indirect', 'risk','evidence','prediction']
tokenized_data = [tokenized_direct, tokenized_indirect, tokenized_risk, tokenized_evidence, tokenized_prediction]

for i in range(5):
    peft_model = get_peft_model(base_model, lora_config)
    output_dir = f'/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/{layer[i]}'
    tokenized_dataset = tokenized_data[i]

    tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.1)

    peft_training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,  # small per-device batch
        gradient_accumulation_steps=20,
        learning_rate=1e-3,
        num_train_epochs=3,
        logging_steps=10
    )

    peft_trainer = Trainer(
        model=peft_model,
        args=peft_training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer
    )

    peft_trainer.train()

    # Save the LoRA adapter
    peft_model.save_pretrained(output_dir)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Step,Training Loss
10,1.172600
20,0.892100
30,0.884000
40,0.874800
50,0.863500
60,0.875800
70,0.814000
80,0.839100
90,0.824200
100,0.846500


<ipython-input-25-fe8ad14531d6>:243: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss
10,1.089500
20,0.923100
30,0.903300
40,0.892700
50,0.889400
60,0.909800
70,0.894500
80,0.874400
90,0.868300
100,0.879900


<ipython-input-25-fe8ad14531d6>:243: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss
10,1.087100
20,0.796400
30,0.784900
40,0.806100
50,0.773400
60,0.765700
70,0.769100
80,0.751400
90,0.749700
100,0.775500


In [ ]:
example = tuneDF.iloc[0].to_dict()  # First example
print("Original example:", example)
tokenized_example = tokenize_function_direct_rev(example)
print("Tokenized example:", tokenized_example)

Original example: {'conversation_id': '86f87e8de3d74e6fa7a011b6cea06399', 'conversation_text': "agent: Hello, thank you for calling Union Financial. How name is Yamilet, how can I assist you today?\nclient: Hi, I'm calling to report a suspicious email that I received. It claims to be from your bank and it asking me my personal information.\nagent: I apologize for any inconvenience this has caused. Can you please provide me with your name and account number so I can verify your identity?\nclient: My name is Tommie, and and my account number is 1234567890.\nagent: Thank you, Mr. Johnson. Unfortunately, the information you provided doesn't match our records. Are you please provide me with some more details to help me verify your identity?\nclient: Okay, well my address is 123 Main Street, and my phone number is 555-555-1234.\nagent: I apologize, but we still cannot verify your identity with the information provided haveve provided. Can you please me the last time digits of your social sec

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import time
import pandas as pd

# Ensure referenceDF is indexed by conversation_id if necessary
# referenceDF = referenceDF.set_index("conversation_id")

# Path to your local LLaMA 3B model
base_model_path = "/content/drive/MyDrive/LLM_Project_Data/llama3.2"

# Load the tokenizer and model once
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
base_model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="cuda")
#base_model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function_direct_rev(example):
    # Construct system and user prompts
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

        Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

        DIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{example["conversation_text"]}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    targets = "DIRECT REVENUE INDICATORS:\n" + example["DIRECT_REVENUE_INDICATORS"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def tokenize_function_indirect_rev(example):
    # Construct system and user prompts
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze indirect revenue indicators. These are more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).

        Identify and extract relevant parts of the text corresponding to these indirect revenue indicators and put them into the format:

        INDIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO INDIRECT REVENUE INDICATORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{example["conversation_text"]}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    targets = "INDIRECT REVENUE INDICATORS:\n" + example["INDIRECT_REVENUE_INDICATORS"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def tokenize_function_risk(example):
    # Construct system and user prompts
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze risk factors. These are elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Identify and extract relevant parts of the text corresponding to these risk factors and put them into the format:

        RISK FACTORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO RISK FACTORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{example["conversation_text"]}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    targets = "RISK FACTORS:\n" + example["RISK_FACTORS"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def tokenize_function_assess(example):
    # Construct system and user prompts
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a summary of a conversation between a bank client and a service agent and assess the evidence provided to assist in predicting future revenue impact.
        Review the provided summary below, which highlights elements of the conversation relevant to three indicators of future revenue:

        1.) Direct Revenue Indicators: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Indicators: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factors: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        The summary will present evidence relevent to these indicators in the format:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        Use the  summary to make an assessment of the weight and reliability of the evidence found related to direct revenue indicators, indirect revenue indicators, and risk factors. Distinguish between explicit statements and implied meanings. Consider the quantity, consistency, and strength of the indicators identified. Put your findings into the format:

        EVIDENCE STRENGTH ASSESSMENT:
        - Direct Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Indirect Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Risk Factor Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        '''
    )
    # The user message includes the conversation text
    user_content = "Here is the summary:\n\n" + "DIRECT REVENUE INDICATORS:\n" + example["DIRECT_REVENUE_INDICATORS"] + "\n\n" + \
                   "INDIRECT REVENUE INDICATORS:\n" + example["INDIRECT_REVENUE_INDICATORS"] + "\n\n" + \
                   "RISK FACTORS:\n" + example["RISK_FACTORS"]

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    targets = "EVIDENCE STRENGTH ASSESSMENT:\n" + example["EVIDENCE_STRENGTH_ASSESSMENT"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def tokenize_function_predict(example):
    # Construct system and user prompts
    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze an assessment of a conversation between a bank client and a service agent to predict future revenue impact.
        Review the provided assessment below, which analyzes evidence relevant to three indicators of future revenue:

        1.) Direct Revenue Evidence: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Evidence: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factor Evidence: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Based on the evidence strength assessment, estimate whether the bank is most likely to see more, less, or the same amount of revenue from that client in the future and the degree of confidence in that revenue trajectory forecast.

        You can think about the revenue trajectories as follows:
        • More Revenue: Clear indicators of expansion, positive satisfaction, life events leading to growth.
        • Same Revenue: Stable relationship, resolved issues, no major changes indicated.
        • Less Revenue: Risk factors present, reduction in services, competitor interest.

        You can think about confidence in your revenue trajectory forecast as follows:
        • High: Multiple consistent indicators across categories.
        • Medium: Some clear indicators but with gaps or inconsistencies.
        • Low: Limited information or conflicting indicators.

        Output your final determination in the following format:

        REVENUE PREDICTION:
        Direction: [More/Same/Less] revenue expected
        Confidence Level: [High/Medium/Low]
        Primary Drivers: [List top 2-3 factors]
        '''
    )
    # The user message includes the conversation text
    user_content = "Here is the assessment:\n\n" + "EVIDENCE STRENGTH ASSESSMENT:\n" + example["EVIDENCE_STRENGTH_ASSESSMENT"]

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    targets = "REVENUE PREDICTION:\n" + example["REVENUE_PREDICTION"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
from datasets import Dataset

train_direct = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "DIRECT_REVENUE_INDICATORS"]])
train_indirect = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "INDIRECT_REVENUE_INDICATORS"]])
train_risk = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text", "RISK_FACTORS"]])
train_evidence = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text",
                                              "DIRECT_REVENUE_INDICATORS",
                                              "INDIRECT_REVENUE_INDICATORS",
                                              "RISK_FACTORS",
                                              "EVIDENCE_STRENGTH_ASSESSMENT"]])
train_prediction = Dataset.from_pandas(tuneDF[["conversation_id", "conversation_text",
                                                 "EVIDENCE_STRENGTH_ASSESSMENT",
                                                 "REVENUE_PREDICTION"]])


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_direct = train_direct.map(tokenize_function_direct_rev, batched=False, remove_columns=train_direct.column_names)
tokenized_indirect = train_indirect.map(tokenize_function_indirect_rev, batched=False, remove_columns=train_indirect.column_names)
tokenized_risk = train_risk.map(tokenize_function_risk, batched=False, remove_columns=train_risk.column_names)
tokenized_evidence = train_evidence.map(tokenize_function_assess, batched=False, remove_columns=train_evidence.column_names)
tokenized_prediction = train_prediction.map(tokenize_function_predict, batched=False, remove_columns=train_prediction.column_names)
# Similarly define tokenize_function_indirect, tokenize_function_risk, etc.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
!pip install peft

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

#print(base_model)
lora_config = LoraConfig(
    r=16, # Rank
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

layer = ['direct', 'indirect', 'risk','evidence','prediction']
tokenized_data = [tokenized_direct, tokenized_indirect, tokenized_risk, tokenized_evidence, tokenized_prediction]

for i in range(5):
  peft_model = get_peft_model(base_model, lora_config)
  output_dir = f'/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters/{layer[i]}'
  tokenized_dataset = tokenized_data[i]

  tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.1)

  peft_training_args = TrainingArguments(
      output_dir=output_dir,
      auto_find_batch_size=True,
      learning_rate=1e-3, # Higher learning rate than full fine-tuning.
      num_train_epochs=1,
      logging_steps=10,
      evaluation_strategy="steps",
      eval_steps=500,
      save_strategy="steps",
      load_best_model_at_end=True,
      metric_for_best_model="eval_loss"
  )

  peft_trainer = Trainer(
      model=peft_model,
      args=peft_training_args,
      train_dataset=tokenized_datasets["train"],
      eval_dataset=tokenized_datasets["test"],
      tokenizer=tokenizer,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
  )

  peft_trainer.train()

  # Save the LoRA adapter
  peft_model.save_pretrained(output_dir)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f70bd7fe0f8d>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,0.797500,0.779228
1000,0.842100,0.772047


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f70bd7fe0f8d>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f70bd7fe0f8d>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss,Validation Loss
500,0.970900,0.953589
1000,0.925300,0.939488


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-f70bd7fe0f8d>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  peft_trainer = Trainer(


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,0.225600,0.216441
1000,0.225600,0.213353


#Evaluate PEFT FTs With Test Set
Now that we have trained and saved down five LoRA adapters that correspond to each step in the prompt, we can evaluate their performance against the test set.

In [ ]:
testDF = test_dataset.to_pandas()
testDF = testDF.merge(truthsDF, on='conversation_id', how='inner')
print(testDF.head())

                    conversation_id  \
0  aee2f63e653049769bda7b0c99ad3b90   
1  9dff2275fe564ab19a4d785f2cb4f7bf   
2  46607d7eb97942eeb48cef087a0dceaf   
3  0952bea524014a8b9a08af4162aea9f8   
4  44f9839702f8459885cac630945ffe21   

                                   conversation_text  \
0  agent: Good morning, thank you for calling Uni...   
1  agent: Good morning, thank you for calling Uni...   
2  agent: Hello morning, thank you for calling Un...   
3  agent: Hello, thank you for holding Union Fina...   
4  agent: Good afternoon, thank you for calling U...   

                                            response  \
0  DIRECT REVENUE INDICATORS:\n- "I'm specificall...   
1  1.) DIRECT REVENUE INDICATORS:\n- "I'm calling...   
2  DIRECT REVENUE INDICATORS:\n- "I was just call...   
3  DIRECT REVENUE INDICATORS:\n- NO DIRECT REVENU...   
4  DIRECT REVENUE INDICATORS:\n- "I need to repor...   

                           DIRECT_REVENUE_INDICATORS  \
0  - "I'm specifically looking for 

In [ ]:
from peft import PeftModel
import re

# For the final REVENUE PREDICTION section
# We'll capture the entire prediction block including direction and other lines.
prediction_pattern = re.compile(r"(?s)REVENUE PREDICTION:\s*(Direction:\s*(More|Same|Less) revenue expected.*)$", re.IGNORECASE)

# Extract the direction separately
direction_pattern = re.compile(r"Direction:\s*(More|Same|Less)", re.IGNORECASE)

# For Direct Revenue Indicators: either multiple lines of '- [Quote ...]' or the 'NO ... FOUND' fallback
direct_regex = re.compile(
    r"DIRECT REVENUE INDICATORS:\s*((?:- \[Quote from transcript\]: [^\n]+\n)+|NO DIRECT REVENUE INDICATORS FOUND)",
    re.IGNORECASE
)

# For Indirect Revenue Indicators
indirect_regex = re.compile(
    r"INDIRECT REVENUE INDICATORS:\s*((?:- \[Quote from transcript\]: [^\n]+\n)+|NO INDIRECT REVENUE INDICATORS FOUND)",
    re.IGNORECASE
)

# For Risk Factors
risk_regex = re.compile(
    r"RISK FACTORS:\s*((?:- \[Quote from transcript\]: [^\n]+\n)+|NO RISK FACTORS FOUND)",
    re.IGNORECASE
)

# For Evidence Strength Assessment
# We'll assume the model outputs exactly one line per evidence category:
evidence_regex = re.compile(
    r"EVIDENCE STRENGTH ASSESSMENT:\s*"
    r"- Direct Revenue Evidence:\s*(Strong|Moderate|Weak)\s*Key points:[^\n]+\n"
    r"- Indirect Revenue Evidence:\s*(Strong|Moderate|Weak)\s*Key points:[^\n]+\n"
    r"- Risk Factor Evidence:\s*(Strong|Moderate|Weak)\s*Key points:[^\n]+",
    re.IGNORECASE
)

def extract_answer_after_marker(generated_text):
    marker = "### OUTPUT START ###"
    pos = generated_text.find(marker)
    if pos == -1:
        # Marker not found, return the full text or handle gracefully
        return generated_text
    # Everything after marker plus the length of the marker
    return generated_text[pos+len(marker):].strip()

results = []
tokenizer.pad_token = tokenizer.eos_token
adapter_paths = {
    "direct": "/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/direct",
    "indirect": "/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/indirect",
    "risk": "/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/risk",
    "evidence": "/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/evidence",
    "prediction": "/content/drive/MyDrive/LLM_Project_Data/PEFT_Adapters2/prediction"
}

first_adapter_name, first_adapter_path = next(iter(adapter_paths.items()))


peft_model = PeftModel.from_pretrained(base_model,
                                       first_adapter_path,
                                       adapter_name=first_adapter_name,
                                       is_trainable=False)

for adapter_name, adapter_path in adapter_paths.items():
    if adapter_name == first_adapter_name:
        continue  # already loaded
    peft_model.load_adapter(adapter_path, adapter_name, is_trainable=False)
peft_model = peft_model.to("cuda")

for idx, row in testDF.iterrows():
    conversation_id = row["conversation_id"]
    convo = row["conversation_text"]



    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze direct revenue indicators. These are explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.

        Identify and extract relevant parts of the text corresponding to these direct revenue indicators and put them into the format:

        DIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO DIRECT REVENUE INDICATORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{convo}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content + "\n\n### OUTPUT START ###\n"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    #start_time = time.time()
    peft_model.set_adapter("direct")

    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    #end_time = time.time()

    #inference_time = end_time - start_time

    # Extract generated tokens count
    output_ids = output[0]


    generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    answer_text = extract_answer_after_marker(generated_text)


    direct_indicators_match = re.search(r"DIRECT REVENUE INDICATORS:\s*(.+)", answer_text, re.IGNORECASE|re.DOTALL)
    if direct_indicators_match:
        direct_indicators_text = "DIRECT REVENUE INDICATORS:\n" + direct_indicators_match.group(1).strip()
    else:
        direct_indicators_text = "DIRECT REVENUE INDICATORS:\n" + "NO DIRECT REVENUE INDICATORS FOUND"


    peft_model.set_adapter("indirect")

    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze indirect revenue indicators. These are more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).

        Identify and extract relevant parts of the text corresponding to these indirect revenue indicators and put them into the format:

        INDIRECT REVENUE INDICATORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO INDIRECT REVENUE INDICATORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{convo}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content+ "\n\n### OUTPUT START ###\n"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    #start_time = time.time()
    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    #end_time = time.time()

    #inference_time += end_time - start_time

    # Extract generated tokens count



    indirect_generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    #print(indirect_generated_text)


    indirect_answer_text = extract_answer_after_marker(indirect_generated_text)


    indirect_indicators_match = re.search(r"INDIRECT REVENUE INDICATORS:\s*(.+)", indirect_answer_text, re.IGNORECASE|re.DOTALL)
    if indirect_indicators_match:
        indirect_indicators_text = "INDIRECT REVENUE INDICATORS:\n" + indirect_indicators_match.group(1).strip()
    else:
        indirect_indicators_text = "INDIRECT REVENUE INDICATORS:\n" + "NO INDIRECT REVENUE INDICATORS FOUND"

    print('extracted text: \n')
    print(indirect_indicators_text)
    print('\n')

    peft_model.set_adapter("risk")

    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a conversation between a bank client and a service agent to assist in predicting future revenue impact.
        Review the conversation transcript below and identify and analyze risk factors. These are elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Identify and extract relevant parts of the text corresponding to these risk factors and put them into the format:

        RISK FACTORS:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        If none are found, output: NO RISK FACTORS FOUND.
        '''
    )
    # The user message includes the conversation text
    user_content = f'Here is the conversation transcript:\n\n{convo}'

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content + "\n\n### OUTPUT START ###\n"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    #start_time = time.time()
    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    #end_time = time.time()

    #inference_time += end_time - start_time

    # Extract generated tokens count
    output_ids = output[0]
    risk_generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    risk_answer_text = extract_answer_after_marker(risk_generated_text)


    risk_match = re.search(r"RISK FACTORS:\s*(.+)", risk_answer_text, re.IGNORECASE|re.DOTALL)
    if risk_match:
        risk_text = "RISK FACTORS:\n" + risk_match.group(1).strip()
    else:
        risk_text = "RISK FACTORS:\n" + "NO RISK FACTORS FOUND"

    print('extracted text: \n')
    print(risk_text)
    print('\n')

    # Combine for evidence step
    combined_summary = f"{direct_indicators_text}\n\n{indirect_indicators_text}\n\n{risk_text}"


    #print(combined_summary)

    peft_model.set_adapter("evidence")

    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a summary of a conversation between a bank client and a service agent and assess the evidence provided to assist in predicting future revenue impact.
        Review the provided summary below, which highlights elements of the conversation relevant to three indicators of future revenue:

        1.) Direct Revenue Indicators: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Indicators: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factors: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        The summary will present evidence relevent to these indicators in the format:
        - [Quote from transcript]: [Brief explanation of relevance]
        -...

        Use the  summary to make an assessment of the weight and reliability of the evidence found related to direct revenue indicators, indirect revenue indicators, and risk factors. Distinguish between explicit statements and implied meanings. Consider the quantity, consistency, and strength of the indicators identified. Put your findings into the format:

        EVIDENCE STRENGTH ASSESSMENT:
        - Direct Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Indirect Revenue Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        - Risk Factor Evidence: [Strong/Moderate/Weak]
          Key points: [Summary]
        '''
    )
    # The user message includes the conversation text
    user_content = f"Here is the summary:\n\n{combined_summary}"
    print(combined_summary)
    print('\n')
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content + "\n\n### OUTPUT START ###\n"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    #start_time = time.time()
    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    #end_time = time.time()

    #inference_time += end_time - start_time

    # Extract generated tokens count
    evidence_generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(evidence_generated_text)
    evidence_answer_text = extract_answer_after_marker(evidence_generated_text)
    print(evidence_answer_text)

    evidence_match = re.search(r"EVIDENCE STRENGTH ASSESSMENT:\s*(.+)", evidence_answer_text, re.IGNORECASE|re.DOTALL)
    if evidence_match:
        evidence_text = "EVIDENCE STRENGTH ASSESSMENT:\n" + evidence_match.group(1).strip()
    else:
        evidence_text = "EVIDENCE STRENGTH ASSESSMENT:\n" + "NO ASSESSMENT FOUND"

    print('extracted text: \n')
    print(evidence_text)
    print('\n')



    system_content = (
        '''
        You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze an assessment of a conversation between a bank client and a service agent to predict future revenue impact.
        Review the provided assessment below, which analyzes evidence relevant to three indicators of future revenue:

        1.) Direct Revenue Evidence: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Evidence: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisfaction, trust, and loyalty; their life circumstances that may alter financial needs (such as moving, career changes, or family events); and the breadth and depth of their existing relationship with the bank (e.g., multiple products or family members also banking at the same institution).
        3.) Risk Factor Evidence: elements of the conversation that suggest a risk to future revenue from the customer. They may include dissatisfaction with the bank’s services, unresolved complaints, interest in or comparison with competitors, concern over fees or rates, or any expressed intent to reduce or discontinue the relationship.

        Based on the evidence strength assessment, estimate whether the bank is most likely to see more, less, or the same amount of revenue from that client in the future and the degree of confidence in that revenue trajectory forecast.

        You can think about the revenue trajectories as follows:
        • More Revenue: Clear indicators of expansion, positive satisfaction, life events leading to growth.
        • Same Revenue: Stable relationship, resolved issues, no major changes indicated.
        • Less Revenue: Meaningful risk factors identified, reduction in services, competitor interest.

        You can think about confidence in your revenue trajectory forecast as follows:
        • High: Multiple consistent indicators across categories.
        • Medium: Some clear indicators but with gaps or inconsistencies.
        • Low: Limited information or conflicting indicators.

        Output your final determination in the following format:

        REVENUE PREDICTION:
        Direction: [More/Same/Less] revenue expected
        Confidence Level: [High/Medium/Low]
        Primary Drivers: [List top 2-3 factors]
        '''
    )
    # The user message includes the conversation text
    user_content = f"Here is the assessment:\n\n{evidence_text}"

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content + "\n\n### OUTPUT START ###\n"}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    #start_time = time.time()
    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=5000,
            temperature=0.0,
            do_sample=False,
        )
    #end_time = time.time()

    #inference_time += end_time - start_time

    # Extract generated tokens count
    prediction_generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print('extracted text: \n')
    print(prediction_generated_text)
    print('\n')

    # Extract final REVENUE PREDICTION
    prediction_match = prediction_pattern.search(prediction_generated_text)
    direction_match = direction_pattern.search(prediction_generated_text)
    predicted_direction = direction_match.group(1) if direction_match else None

    # Ground truth direction from your dataset
    ground_truth_direction = row["REVENUE_DIRECTION"]

    results.append({
        "conversation_id": conversation_id,
        "predicted_direction": predicted_direction,
        "ground_truth_direction": ground_truth_direction,
        "full_output": prediction_generated_text
    })
    print("Conclusion: \n")
    print(predicted_direction)
    print(ground_truth_direction)
    print('\n')



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


extracted text: 

INDIRECT REVENUE INDICATORS:
- "I'm specifically looking for any suspicious activity that may have occurred within the past six months.": This indicates a level of engagement and concern for account security, which may suggest that the client values the relationship with the bank and is likely to continue using their services.
- "I recommend that you review keep a close eye on your account activity and report any other suspicious transactions to us immediately.": This reflects the bank's commitment to customer service and security, which can enhance customer loyalty and trust.




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


extracted text: 

RISK FACTORS:
- "I didn't authorize any of those transactions.": This indicates a potential risk to the relationship as it suggests dissatisfaction with the bank's security measures and could lead to a loss of trust.
- "I recommend that you review keep a close eye on your account activity and report any other suspicious transactions to us immediately.": This implies that the client may be concerned about their account's security, which could lead to a negative perception of the bank if not addressed properly.


DIRECT REVENUE INDICATORS:
- "I'm specifically looking for any suspicious activity that may have occurred within the past six months.": This indicates the client is actively monitoring their account, which may lead to further engagement with the bank's services if they find issues that require resolution.
- "I'd appreciate it if you could provide me with more details about those transactions.": The client's request for more information suggests a desire to unde

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

extracted text: 

INDIRECT REVENUE INDICATORS:
- "Thanks for your help, Drew!": This expression of gratitude suggests a positive customer experience, which can lead to increased loyalty and potential future business.
- "It was my pleasure. Have a great day!": The friendly interaction between the agent and client may enhance customer satisfaction and loyalty, indicating a potential for continued business.




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


extracted text: 

RISK FACTORS:
- NO RISK FACTORS FOUND.


DIRECT REVENUE INDICATORS:
- "I'm interested in learning more about your interest rates and finance options.": This indicates the client's interest in exploring new banking products, which could lead to increased revenue for the bank.
- "I think I'll go with the certificate of deposit. Can you help me set it up?": The client's decision to open a certificate of deposit (CD) represents a direct revenue opportunity through the bank's interest income from the client's deposited funds.

INDIRECT REVENUE INDICATORS:
- "Thanks for your help, Drew!": This expression of gratitude suggests a positive customer experience, which can lead to increased loyalty and potential future business.
- "It was my pleasure. Have a great day!": The friendly interaction between the agent and client may enhance customer satisfaction and loyalty, indicating a potential for continued business.

RISK FACTORS:
- NO RISK FACTORS FOUND.




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


system

You are a skilled financial analyst specializing in customer behavior analysis. Your task is to analyze a summary of a conversation between a bank client and a service agent and assess the evidence provided to assist in predicting future revenue impact.
        Review the provided summary below, which highlights elements of the conversation relevant to three indicators of future revenue:

        1.) Direct Revenue Indicators: explicit signals in the conversation that directly affect the customer’s future revenue contribution. Examples include mentions of opening or closing accounts, expressing interest in new banking products or services, changing transaction volumes or values, or altering usage patterns that have a clear, measurable impact on revenue.
        2.) Indirect Revenue Indicators: more subtle or contextual factors that can influence the customer’s future revenue potential but are not directly tied to a product or transaction. This might include their overall satisf

#Below are the results of the second and final experiment. Note the improvement over the initial one (below). This indicates that further training could refine/improve performance.

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("/content/drive/MyDrive/LLM_Project_Data/ft_performance2.csv", index=False)

# Calculate accuracy
correct = (results_df["predicted_direction"] == results_df["ground_truth_direction"]).sum()
total = len(results_df)
accuracy = correct / total if total > 0 else 0.0
print(accuracy)
from sklearn.metrics import classification_report

y_true = results_df["ground_truth_direction"]
y_pred = results_df["predicted_direction"]

print(classification_report(y_true, y_pred, labels=["More", "Same", "Less"], zero_division=0))

0.55
              precision    recall  f1-score   support

        More       0.88      0.72      0.79        29
        Same       1.00      0.12      0.21        42
        Less       0.41      1.00      0.58        29

    accuracy                           0.55       100
   macro avg       0.76      0.61      0.53       100
weighted avg       0.79      0.55      0.49       100



In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("/content/drive/MyDrive/LLM_Project_Data/ft_performance.csv", index=False)

# Calculate accuracy
correct = (results_df["predicted_direction"] == results_df["ground_truth_direction"]).sum()
total = len(results_df)
accuracy = correct / total if total > 0 else 0.0
print(accuracy)

0.51


#Below are the results of the first run relative to the test set


In [ ]:
from sklearn.metrics import classification_report

y_true = results_df["ground_truth_direction"]
y_pred = results_df["predicted_direction"]

print(classification_report(y_true, y_pred, labels=["More", "Same", "Less"], zero_division=0))

              precision    recall  f1-score   support

        More       0.86      0.66      0.75        29
        Same       1.00      0.07      0.13        42
        Less       0.39      1.00      0.56        29

    accuracy                           0.51       100
   macro avg       0.75      0.58      0.48       100
weighted avg       0.78      0.51      0.43       100

